In [142]:
# Import libraries

import pandas as pd
import numpy as np
import math 
import rbo
import scipy.stats as stats
import sys


import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None




In [89]:
# Read the data, I give some format and the name for each column

df1 = pd.read_csv('/Users/alejandroospina/Desktop/1. USI/4. IV SEMESTER/1. Thesis/6. TREC CAsT 2022 - MI runfiles/baseline_prf_mi.trec', sep=' ', names=["qid", "Q0", "docno", "rank", "score", "tag"])
df2 = pd.read_csv('/Users/alejandroospina/Desktop/1. USI/4. IV SEMESTER/1. Thesis/6. TREC CAsT 2022 - MI runfiles/baseline_prf.trec', sep=' ', names=["qid", "Q0", "docno", "rank", "score", "tag"])


df1


In [90]:
# export doc in csv file to better see the index for next queries 

df1.to_csv('/Users/alejandroospina/Desktop/1. USI/4. IV SEMESTER/1. Thesis/6. TREC CAsT 2022 - MI runfiles/df1.csv')

df2.to_csv('/Users/alejandroospina/Desktop/1. USI/4. IV SEMESTER/1. Thesis/6. TREC CAsT 2022 - MI runfiles/df2.csv')

In [33]:
# Missing data in df1?

for col in df1.columns:
    pct_missing = np.mean(df1[col].isnull())
    print('{} - {}%'.format(col, pct_missing))
    

qid - 0.0%
Q0 - 0.0%
docno - 0.0%
rank - 0.0%
score - 0.0%
tag - 0.0%


In [50]:
# Missing data in df2?

for col in df2.columns:
    pct_missing = np.mean(df2[col].isnull())
    print('{} - {}%'.format(col, pct_missing))

qid - 0.0%
Q0 - 0.0%
docno - 0.0%
rank - 0.0%
score - 0.0%
tag - 0.0%


In [113]:
M = df1['docno'].tolist()

N = df2['docno'].tolist()


In [118]:
# Kendall tau--> It is possible to apply the Kendall Tau? I have to check if this list have the same element (are conjoint). If not, I can't apply kendall

# https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.kendalltau.html
# https://towardsdatascience.com/rbo-v-s-kendall-tau-to-compare-ranked-lists-of-items-8776c5182899
# Negative point: It is unweighted i.e., it places as much emphasis on the disagreement at the bottom of the list as much as the top (in popular search engines, the results in the “head” matter a lot more than the results in the “tail”)

# Here I'm selecting the documents of the firts query of both file. The query id is 132_1-1
S = M[:999]
T = N[:999]


sorted(S) == sorted(T)

# As the result is False, I can't. 

False

In [119]:
# Rank Biased Overlap (RBO)
# https://towardsdatascience.com/how-to-objectively-compare-two-ranked-lists-in-python-b3d74e236f6a


def rbo(S,T, p= 0.9):
    """ Takes two lists S and T of any lengths and gives out the RBO Score
    Parameters
    ----------
    S, T : Lists (str, integers)
    p : Weight parameter
    
    Returns
    -------
    Float of RBO score
    """
    
    # Fixed Terms
    k = max(len(S), len(T))
    x_k = len(set(S).intersection(set(T)))
    
    summation_term = 0

    # Loop for summation
    # k+1 for the loop to reach the last element (at k) in the bigger list    
    for d in range (1, k+1): 
            # Create sets from the lists
            set1 = set(S[:d]) if d < len(S) else set(S)
            set2 = set(T[:d]) if d < len(T) else set(T)
            
            # Intersection at depth d
            x_d = len(set1.intersection(set2))

            # Agreement at depth d
            a_d = x_d/d   
            
            # Summation
            summation_term = summation_term + math.pow(p, d) * a_d

    # Rank Biased Overlap - extrapolated
    rbo_ext = (x_k/k) * math.pow(p, k) + ((1-p)/p * summation_term)

    return rbo_ext

In [120]:
print(rbo(S,T))
# as the result is 0.76, there is a good correlation but there aren't the same time of document retrieved for the same query 

0.7577571548087627


In [132]:
# define the weight 


def weightage_calculator(p,d):
    

    summation_term = 0

    for i in range (1, d): # taking d here will loop upto the value d-1 
        summation_term = summation_term + math.pow(p,i)/i


    Wrbo_1_d = 1 - math.pow(p, d-1) + (((1-p)/p) * d *(np.log(1/(1-p)) - summation_term))

    return Wrbo_1_d

In [136]:
# the first 5 documents are normally the most important for the user. 
# for that I give more weight

weightage_calculator(0.9,10)

# the result is 0.67, means that the correlation between this first 5 documents, begin to low 
# If I take the first 10, the result will be better (0.85)



0.8555854467473518

In [137]:
# the comparison score 
rbo (S, T, 0.76)

# the result is 0.89. Means that are T is very similar to S

0.8942489396943707

In [140]:
d = {'Q1':S,'Q2':T}
new_df = pd.DataFrame(d)
new_df

,Q1,Q2
0,MARCO_33_1621342262-1,MARCO_33_1621342262-1
1,MARCO_30_1605964568-1,MARCO_30_1605964568-1
2,MARCO_27_1574256585-1,MARCO_27_1574256585-1
3,MARCO_33_1621342262-3,MARCO_33_1621342262-3
4,MARCO_04_643170393-2,MARCO_57_1276848320-4
5,MARCO_06_1532160688-6,MARCO_30_1605964568-5
6,MARCO_30_1605964568-8,MARCO_16_3117875026-1
7,MARCO_30_1605964568-6,MARCO_30_1605964568-8
8,MARCO_30_1605964568-5,MARCO_04_643170393-2
9,MARCO_30_1605964568-2,MARCO_33_1621342262-2


In [161]:
# I want to know which and how many documents are different between the first query of both file.

c1 = Counter(S[:10])
c2 = Counter(T[:10])
diff = list((c1 - c2).elements())

print("No of elements that are different:", len(diff))
print(diff)



No of elements that are different: 3
['MARCO_06_1532160688-6', 'MARCO_30_1605964568-6', 'MARCO_30_1605964568-2']
